In [1]:
from collections import OrderedDict
import torch
from torchsummary import summary
import torch.nn as nn
import numpy as np
from torch.nn import functional as F
from collections import OrderedDict

In [2]:
import os

import torch
import numpy as np
import scipy.io as scio
from torch.utils.data import Dataset, DataLoader
import random

trainfolder='/datadisk/sdc/wangruibo/maml_e2e/'
testfolder='/datadisk/sdc/wangruibo/maml_e2e/'

class dataset(Dataset):

    def __init__(self,input_path, target_path, mode = 'train', seedNN = 0):
        super(dataset, self).__init__()
        self.input_data = []
        self.target_data = []
        input_data_meta = np.load(input_path)
        target_data_meta = np.load(target_path)
        self.length = input_data_meta.shape[0]
        
#         random.seed(seedNN)
#         ind_shuffle = random.sample(range(0, self.length), self.length)
        # random seed inside
        if mode == 'train':
            self.target_data = torch.randint(0, 2, (10000,72*14, 4))
            self.input_data = input_data_meta
#             self.input_data = input_data_meta[:int(0.2 * self.length)]
        else:
            self.target_data = torch.randint(0, 2, (10000,72*14, 4))
#             self.input_data = input_data_meta[int(0.2 * self.length):]
            self.input_data = input_data_meta
        self.length = self.input_data.shape[0]
    def __len__(self):
        return self.length

    def __getitem__(self, index):
#         print(self.input_data[index].shape)
        input_data = torch.from_numpy(self.input_data[index])
        target_data = self.target_data[index]
        return input_data, target_data

class UmaDatasets(Dataset):
    def __init__(self,trainFolder=trainfolder,testFolder=testfolder):
       # number of tasks
        self.length=0
        self.traindataloader_Loc={}
        self.testdataloader_Loc={}
#         self.Channel_idx=['140-200-low', '140-200-medium', '140-200-high'
#                             , '20-80-low','20-80-medium','20-80-high',
#                            '80-140-low','80-140-medium','80-140-high']
        self.Channel_idx=['140-200-low', '140-200-medium', '140-200-high']
#         self.Channel_idx=['20-80-low','20-80-medium','20-80-high',
#                            '80-140-low','80-140-medium','80-140-high']
        self.strength_range= ['0.0','0.66','1.33','2.0']  
        self.TrainDataFolder=trainFolder
        self.TestDataFolder=testFolder
        self.load()
        
    def load(self):
#         print('Train Data Folder: ',self.TrainDataFolder)
        for il in self.Channel_idx:
            if il.split('-')[2] == 'low':
                m,n = self.strength_range[0],self.strength_range[1]
            elif il.split('-')[2] == 'medium':
                m,n = self.strength_range[1],self.strength_range[2]
            else:
                m,n = self.strength_range[2],self.strength_range[3]
#             print(il)
            input_path11 = self.TrainDataFolder + il + '/dataset_UMa_D={}-{}_&_A={}-{}_s=10000.npy'.format(float(il.split('-')[0]), float(il.split('-')[1]),m,n)
            target_path1 = self.TrainDataFolder + 'bit_labels_s_size=10000.npy'
#             print(il,' ',len(input_path11))
#             print(len(target_path1))
            trainset= dataset(input_path11, target_path1, mode='train', seedNN = 0)
            
            self.length+=trainset.length
            self.traindataloader_Loc[il]= trainset
            
#             self.TestDataFolder
            input_path21 = self.TrainDataFolder + il + '/dataset_UMa_D={}-{}_&_A={}-{}_s=10000.npy'.format(float(il.split('-')[0]), float(il.split('-')[1]),m,n)
            target_path2 = self.TrainDataFolder + 'bit_labels_s_size=10000.npy'
            testset    = dataset(input_path21, target_path2, mode='test', seedNN = 0)
            self.testdataloader_Loc[il]= testset
            
    def sample_task(self, mode='train'):
        return np.random.choice(self.Channel_idx)
    def sample_datas(self, name, K = 10, mode='train'):
        data=[]
        label=[]
        if mode == 'train':
            tidx=np.random.choice(range(len(self.traindataloader_Loc[name])),K)
            for i in tidx:
                data.append(self.traindataloader_Loc[name][i][0])
                label.append(self.traindataloader_Loc[name][i][1])
        else:
            tidx=np.random.choice(range(len(self.testdataloader_Loc[name][0])),K)
#             datas=self.testdataloader_Loc[name][0]
#             labels=self.testdataloader_Loc[name][1]
            for i in tidx:
                data.append(self.testdataloader_Loc[name][i][0])
                label.append(self.testdataloader_Loc[name][i][1])
        return data,label
    
    def __len__(self):
        return self.length

In [3]:
uma_dataset = UmaDatasets(trainfolder,testfolder)

In [4]:
def gen_labeling(bit_per_cu):
    
        def genCode(n):
                if n == 0:
                    return ['']

                code1 = genCode(n-1)
                code2 = []
                for codeWord in code1:
                    code2 = [codeWord] + code2

                for i in range(len(code1)):
                    code1[i] += '0'
                for i in range(len(code2)):
                    code2[i] += '1'
                return code1 + code2  

        k = bit_per_cu // 2 # bits per dimension
        syms = 2**k
        m = syms**2
        # gray labeling
        gray_labeling = []
        for i in range(2**k):
            gray_labeling.append(np.array([int(e) for e in  genCode(k)[i]]))
        gray_labeling = np.array(gray_labeling)

        encoding_table_256_qam = gray_labeling[:,::-1]

        encoding_table_256_qam = np.hstack( [np.kron(encoding_table_256_qam,np.ones((syms,1))), np.tile(encoding_table_256_qam,(syms,1)) ])

        natural_number = np.sum(2**np.arange(2*k)[::-1] * encoding_table_256_qam,axis=1)
        table_gray = np.zeros((m,m))
        for bvec in range(m):
            table_gray[bvec,np.where(natural_number==bvec)[0]] =1

        return table_gray

def gen_const(bit_per_cu):
    
    points_per_dim = 2**(bit_per_cu // 2)
    
    A = np.arange(0, points_per_dim)
    A = A - np.mean(A)
    
    C = []
    for i in A:
        for j in A:
            C.append([i,j])
    C = np.array(C)
    
    L = gen_labeling(bit_per_cu)
    L = np.argmax(L, axis=1)
    C = C[L]
    
    return C


class GS(nn.Module):
    def __init__(self,  bit_per_cu, init_qam=False):
        super(GS, self).__init__()
        self.bit_per_cu = bit_per_cu
        self.init_qam = init_qam
        if self.init_qam:
            C = gen_const(self.bit_per_cu)
            en_point = np.mean(np.sum(np.square(C), axis=1))
            C = C / np.sqrt(en_point)
            self.Cr = torch.tensor(C[:,0],dtype = torch.float())#,requires_grad=True)
            self.Ci = torch.tensor(C[:,1],dtype = torch.float())#,requires_grad=True)
            self.Cr = nn.Parameter(self.Cr)
            self.Ci = nn.Parameter(self.Ci)
        else:
            const_size = 2**self.bit_per_cu
            self.Cr = torch.normal(mean = 0.0,std=0.05,size = (1,const_size)).squeeze()#.requires_grad_(True)
            self.Ci = torch.normal(mean = 0.0,std=0.05,size = (1,const_size)).squeeze()#.requires_grad_(True)
            self.Cr = nn.Parameter(self.Cr)
            self.Ci = nn.Parameter(self.Ci)
    def get_const(self):
        C = torch.complex(self.Cr, self.Ci)
        # Center the constellation
        C = C - torch.mean(C)
        
        # Normalization
        en_point = torch.mean(torch.square(torch.real(C)) + torch.square(torch.imag(C)))
        norm_fac = torch.complex(torch.sqrt(en_point), torch.tensor(0.0).cuda())
        C = torch.div(C, norm_fac)
        if np.inf in C:
            C[C == np.inf] = torch.tensor(0.0).cuda()
        
        return C
    def forward(self,s):
        
        batch_size = s.shape[0]
        C = self.get_const()
        C = torch.tile(torch.unsqueeze(C, axis=0), (batch_size, 1))
                
        # Mapping
        s = s.long()
        x = torch.gather(C,1,s)
        
        return x


# class SeparableConv2d(torch.nn.Module):
#     def __init__(self, 
#                  in_channels,
#                  out_channels,
#                  kernel_size,
#                  dilation_size,
#                  padding_size,
#                  bias=True
#         ):
#         super(SeparableConv2d,self).__init__()
#         self.spatialConv = convnxn(
#              in_channels=in_channels,
#              out_channels=out_channels,
#              kernel_size=kernel_size,
#              padding_size=padding_size,
#              dilation_size=dilation_size,
#              groups_size=in_channels
#         )
#         self.pointConv = convnxn(
#              in_channels=in_channels,
#              out_channels=out_channels,
#              kernel_size=1,
#              padding_size=0,
#              dilation_size=1,
#              groups_size = 1
#         )
    
#     def forward(self, x, params=None, episode=None):
#         return self.pointConv(self.spatialConv(x, get_child_dict(params, 'spatialConv')), get_child_dict(params, 'pointConv'))


class ResNetBlock(nn.Module):
    
    def __init__(self, filters, kernel_size,dilation_rate, use_dense,padding_size,training):
        super(ResNetBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.use_dense = use_dense
        self.padding_size = padding_size
        self.training = training
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        if self.use_dense:
            self.dense = nn.Linear(self.filters,self.filters,bias = False)
        self.bn1 = nn.BatchNorm2d(self.filters, affine = self.training)
        self.conv1 = nn.Conv2d(self.filters,self.filters, self.kernel_size, dilation=self.dilation_rate, padding=self.padding_size,groups = 1)
        self.bn2 = nn.BatchNorm2d(self.filters, affine = self.training)
        self.conv2 = nn.Conv2d(self.filters,self.filters, self.kernel_size, dilation=self.dilation_rate, padding=self.padding_size,groups = 1)
    
    def forward(self, x):
        z = x
        
        z = self.bn1(z)
        z = self.relu1(z)
        z = self.conv1(z)
        z = self.bn2(z)
        z = self.relu2(z)
        z = self.conv2(z)
#         z = self.bn1(z)
#         z = self.relu1(z)
#         z = self.conv1(z)
#         z = self.bn2(z)
#         z = self.relu2(z)
#         z = self.conv2(z)
        if self.use_dense:
            y = self.dense(x)
        else:
            y = x
        z = z + y
        
        return z

    
class OFDM:
    
    # num_subc: number of subcarriers
    # cp_length: Cycle prefix duration [symbols]
    # use_cp: If True, uses a cycling-prefix, otherwise does not.
    def __init__(self, num_subc, cp_length, use_cp=True):
        
        self.num_subc = num_subc
        self.cp_length = cp_length
        self.use_cp = use_cp
    
    # x_f: Baseband symbols in the frequency domain [batch size, number of subcarriers, number of OFDM symbols]
    # The ordering of the subcarriers is assumed to be as follows: negatve frequencies (in order of decreasingly negative frequency),
    #  then center frequency, then positive frequencies.
    def modulate(self, x_f):
        
        # Checking the number of subcarriers is large enough given the CP length
        # tf.debugging.assert_positive(tf.shape(x_f)[1] - self.cp_length, "Number of subcarriers too small given the specified CP length")
        
        x_f = torch.transpose(x_f, 2, 1)
        
        # FFT shift
        x_f = torch.fft.ifftshift(x_f, dim=2)
        
        # Moving to time domain
        x = torch.fft.ifft(x_f)*torch.complex(torch.sqrt(torch.tensor(x_f.shape[-1])), torch.tensor(0.0))
        
        # Adding CP
        if self.use_cp:
            x = torch.cat((x[:,:,-self.cp_length:], x), dim=2)
        
        # Reshaping to get a single time domain signal
        x = x.view(x.shape[0], -1)
        
        return x
        
    
    # y_t: Baseband symbols in the time domain [batch size, time steps]
    def demodulate(self, y_t):
        
        if self.use_cp:
            symbol_length = self.num_subc + self.cp_length
        else:
            symbol_length = self.num_subc
        
        # Checking the signal dimension matches the specified CP length and number of subcarriers
        num_symbs = torch.div(y_t.shape[1], symbol_length)
        rem_symbs = torch.fmod(torch.tensor(y_t.shape[1]), symbol_length)
#         tf.debugging.assert_positive(num_symbs, "Signal duration is too short given specified OFDM configuration")
#         tf.debugging.assert_equal(rem_symbs, 0, "Signal duration does not match specified OFDM configuration")
        
        # Reshaping
        y = y_t.view(y_t.shape[0], -1, symbol_length)
        
        # Removing CP
        if self.use_cp:
            y = y[:,:,self.cp_length:]
        
        # Moving to frequency domain
        y = torch.fft.fft(y)/torch.complex(torch.sqrt(torch.tensor(y.shape[-1])), torch.tensor(0.0))
        
        # FFT shift
        y = torch.fft.fftshift(y, dim=2)
        
        y = torch.transpose(y, 2, 1)
        
        return y
    

class MultiPathChannel(nn.Module):
    
    def __init__(self):
        super(MultiPathChannel, self).__init__()
         
    # x: time domain baseband channel input symbols, with shape [batch_size, number of time steps]
    # h: channel response in time domain, with shape [batch_size, number of time steps, number of taps]
    # noise_power_db: Noise power in dB. Scalar
    def forward(self, x, h, noise_power_db):
        
        batch_size = x.shape[0]
        time_steps_no = x.shape[1]
        taps_no = h.shape[2]
        
        # If extra time steps are available for the channel, we just ignore them
        h = h[:,:time_steps_no,:]
        # Reshaping the channel input symbols
        x = torch.unsqueeze(x, dim=2)
        x = torch.tile(x, (1, 1, taps_no))
        padding_r = torch.zeros((batch_size, taps_no, taps_no))
        padding_i = torch.zeros((batch_size, taps_no, taps_no))
        padding = torch.complex(padding_r,padding_i)
        padding = padding
        x = torch.cat((x, padding.cuda()), axis=1)
        x_ = torch.transpose(torch.zeros_like(torch.transpose(torch.zeros_like(x), 0,2)), 1,2)
        for i in range(taps_no):
            z = x[:,:,i]
            z = torch.roll(z, i, 1)
            x_[i] = z
        x_ = torch.transpose(x_, 0,1)
        x_ = torch.transpose(x_, 1,2)
        x = x_[:,:time_steps_no,:] # [batch size, number of timer steps, number of taps]
        # Channel transfer function
        y = torch.sum(h*x, axis=2)
        noise_std = torch.sqrt(0.5*(torch.pow(10.0, noise_power_db/10.0)))
        noise_std = torch.unsqueeze(noise_std, dim=1)
        noise_std = noise_std.cpu().numpy()
        noise = (np.random.normal(0.0, noise_std,size = (y.shape[0],y.shape[1])), np.random.normal(0.0, noise_std,size = (y.shape[0],y.shape[1])))
        noise = torch.complex(torch.tensor(noise[0]), torch.tensor(noise[1]))
        noise = noise.cuda()
        y = y + noise
        y = y.to(dtype=torch.complex64)
        return y

    
class RXNN(nn.Module):
    
    def __init__(self, bit_per_cu,training):
        super(RXNN, self).__init__()
        
        self.bit_per_cu = bit_per_cu
        self.training = training
        self.conv_in = nn.Conv2d(5,256, (3,3), dilation=(1,1), padding=(1,1))#padding 1,1
        self.res1 = ResNetBlock(256, (3,3), (3,1),False,(3,1),training)#padding 3,1
        self.res2 = ResNetBlock(256, (3,3), (6,2), False,(6,2),training)#padding 6,2
        self.res3 = ResNetBlock(256, (3,3), (6,2), False,(6,2),training)#padding 6,2
        self.res4 = ResNetBlock(256, (3,3), (3,1), False,(3,1),training)#padding 3,1
        self.conv_out = nn.Conv2d(256,4, (1,1), dilation=(1,1), padding=(0,0))#padding 0,0
    
    # all inputs of shape [nm subframes, num subcarriers, block length]
    # y: received signal
    # snr_db: SNR [dB]
    def forward(self, y, snr_db):
        
        # Positional encoding
        num_frames = y.shape[0]
        num_subc = y.shape[1]
        block_length = y.shape[2]
        spectral_pos_enc = torch.linspace(0.0, 1.0, num_subc)
        time_pos_enc = torch.linspace(0.0, 1.0, block_length)
        spectral_pos_enc = torch.unsqueeze(spectral_pos_enc, dim =1)
        spectral_pos_enc = torch.tile(spectral_pos_enc, (1, block_length))
        time_pos_enc = torch.unsqueeze(time_pos_enc, axis=0)
        time_pos_enc = torch.tile(time_pos_enc, (num_subc, 1))
        pos_enc = torch.stack([spectral_pos_enc, time_pos_enc], dim =2)
        pos_enc = torch.unsqueeze(pos_enc, axis=0)
        pos_enc = torch.tile(pos_enc, (num_frames, 1, 1, 1))

        #         # Inputs
        snr_db = snr_db
        pos_enc = pos_enc.cuda()
        z = torch.stack((torch.real(y), torch.imag(y),snr_db),dim = 3)
        z = torch.cat((z, pos_enc), dim=3)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 2, 3)
        # Call conv layers
#         print(z)
        z = z.float()
        z = self.conv_in(z)
        z = self.res1(z)
        z = self.res2(z)
        z = self.res3(z)
        z = self.res4(z)
        z = self.conv_out(z)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 1, 2)
        return z


class SysGS_torch_gs(nn.Module):
    def __init__(self):
        super(SysGS_torch_gs, self).__init__()
        
        self.gs = GS(4)
        self.ofdm = OFDM(72, 6, False)
        self.channel = MultiPathChannel()
        self.rx = RXNN(4,True)
    def forward(self,snr_db,h,s,params=None, episode=None):
        noise_power_db = -snr_db
#         print(s.shape)
        x = self.gs(s)
        x = x.view(10,72,14)
        x = self.ofdm.modulate(x)
        y = self.channel(x,h,noise_power_db)
        y = self.ofdm.demodulate(y)
        snr_db_rx = torch.tile(torch.unsqueeze(torch.unsqueeze(snr_db, dim=1), dim=2), [1, 72, 14])
        y = self.rx(y,snr_db_rx)
        return y

In [5]:
net = SysGS_torch_gs()

In [7]:
import torch.optim as optim
import time
def e2e_traninig(net,Epoch,dataset,SaveFolder,savename,K,maml = True,evaluation=False,save_num = 500,inner_loop = 50):
    
    # Construct NN and parameters
#     net = SubCAN(32, 3, 6, 8).cuda()
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in net.parameters()))
    print(NetworkParamNum)
    
    # Construct scheduler
    lr = 0.00001
    # criterion = nn.L1Loss()
    # optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=10**(-5))
    optimizer = optim.Adam(net.parameters(), lr=lr)
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    
    # Initialize some parameters
    min_testloss = 100
    testError  = []
    trainError = []
    costTime   = []
    BER_train = []
    BER_test = []
    KL_train = []
    KL_test = []
    storeStepResult = True
    minloss = 1000000000
    
    for epoch in range(Epoch):
        net.train()
        train_loss = 0.0
        test_loss  = 0.0      
        numTrain = 0
        numTest  = 0 
        bertrain = 0
        bertest = 0
        time_start = time.time()
        task = dataset.sample_task()
        for i in range(1,inner_loop):
            optimizer.zero_grad()
            X, y = dataset.sample_datas(task, K)
            X = torch.stack(X).float().cuda()
            y = torch.stack(y).float()
            snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 25.0).squeeze().cuda()
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(y.cpu()*A, axis=2)
            s = s.cuda()
            y = y.cuda()

            t_pred = net(snr_db,X,s)
            LLR = t_pred
            LLR = torch.where(LLR >= 0, 1,-1)
            LLR = (LLR+1)/2
            LLR = LLR.view(LLR.shape[0],LLR.shape[1]*LLR.shape[2],LLR.shape[3])
            bertrain += (1-((LLR==y).sum().item()/(LLR.shape[0]*LLR.shape[1]*LLR.shape[2])))
            #             print(t_pred.shape)
            t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            loss = F.binary_cross_entropy_with_logits(t_pred,y)
                            # loss = criterion(t_pred, target)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
#         scheduler.step()
            
        
        numTrain = 50
                    
            # 测试
        if evaluation:
            net.eval()
            task = dataset.sample_task()
            X, y = dataset.sample_datas(task, K)
            X = torch.stack(X).float().cuda()
            y = torch.stack(y).float()
            snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 25.0).squeeze().cuda()
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(y.cpu()*A, axis=2)
            s = s.cuda()
            y = y.cuda()

            t_pred = net(snr_db,X,s)
            LLR = t_pred
            LLR = torch.where(LLR >= 0, 1,-1)
            LLR = (LLR+1)/2
            LLR = LLR.view(LLR.shape[0],LLR.shape[1]*LLR.shape[2],LLR.shape[3])
            bertest += (1-((LLR==y).sum().item()/(LLR.shape[0]*LLR.shape[1]*LLR.shape[2])))
        #             print(t_pred.shape)
            t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            loss = F.binary_cross_entropy_with_logits(t_pred,y)
#                 KL = KL(t_pred,target)
                        # loss = criterion(t_pred, target)
            test_loss += float(loss.item())
            numTest = 1
        
        time_end = time.time()
        
        train_loss /= numTrain
        bertrain /= numTrain
#         train_loss /= torch.log(torch.tensor([2.]))
#         train_loss = torch.mean(torch.tensor(4.)-torch.sum(train_loss,axis = 3))
        if evaluation:
            test_loss  /= numTest
            bertest /= numTest
            
#         test_loss /= torch.log(torch.tensor([2.]))
        # For print and record Loss
        cost = time_end - time_start
        if evaluation:
            showStrings = '[Epoch: %3d/%3d][Ber train:%3.5f][Ber test:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, bertrain,bertest, cost)
        else:
            showStrings = '[Epoch: %3d/%3d][Ber:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, bertrain, cost)
        print(showStrings, end='\n')
        if maml:
            name = 'MAML' + savename
        else:
            name = 'Joint' + savename
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        BER_train.append(bertrain)
        BER_test.append(bertest)
        file = open(SaveFolder + name + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.write('BER train: ' + str(BER_train) + "\n")
        file.write('BER test: ' + str(BER_test) + "\n")
        file.close()
        if epoch % save_num == 0:
            torch.save(net.state_dict(),  SaveFolder+ 'Checkpoints-Epk{}.pth'.format(epoch))

In [8]:
e2e_traninig(net.cuda(),10000,uma_dataset,"/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/JOINT_ORIGINIAL_5000/",'Checkpoints',10,maml = False,evaluation=True)

Total number of paramerters in networks is 4737572  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)


[Epoch:   1/10000][Ber train:0.50045][Ber test:0.49859][CostTime: 4.24747 s]
[Epoch:   2/10000][Ber train:0.49973][Ber test:0.50107][CostTime: 3.98205 s]
[Epoch:   3/10000][Ber train:0.49978][Ber test:0.50216][CostTime: 4.03739 s]
[Epoch:   4/10000][Ber train:0.49938][Ber test:0.50275][CostTime: 4.05734 s]
[Epoch:   5/10000][Ber train:0.50009][Ber test:0.49898][CostTime: 2.87943 s]
[Epoch:   6/10000][Ber train:0.49963][Ber test:0.50345][CostTime: 2.86074 s]
[Epoch:   7/10000][Ber train:0.49970][Ber test:0.49692][CostTime: 2.91615 s]
[Epoch:   8/10000][Ber train:0.49914][Ber test:0.49683][CostTime: 2.87904 s]
[Epoch:   9/10000][Ber train:0.49935][Ber test:0.50092][CostTime: 2.86212 s]
[Epoch:  10/10000][Ber train:0.49915][Ber test:0.49836][CostTime: 2.87226 s]
[Epoch:  11/10000][Ber train:0.49920][Ber test:0.49591][CostTime: 2.89942 s]
[Epoch:  12/10000][Ber train:0.49936][Ber test:0.49921][CostTime: 2.86285 s]
[Epoch:  13/10000][Ber train:0.49913][Ber test:0.49618][CostTime: 2.86246 s]

In [9]:
class scratch_dataset(Dataset):

    def __init__(self,input_path, target_path, mode = 'train', seedNN = 0):
        super(scratch_dataset, self).__init__()
        self.input_data = []
        self.target_data = []
        input_data_meta = []
        target_data_meta = []
        self.mode_t = mode
        for i in input_path:
            input_data_meta += list(np.load(i))
        for i in target_path:
            target_data_meta += list(np.load(i))
        input_data_meta = np.array(input_data_meta)
        target_data_meta = np.array(target_data_meta)
        self.length = input_data_meta.shape[0]
        random.seed(seedNN)
        ind_shuffle = random.sample(range(0, self.length), self.length)
        if mode == 'train':
            self.target_data = target_data_meta
            self.input_data = input_data_meta
#             self.input_data = np.reshape(self.input_data,(int(self.input_data.shape[0]/10),10,self.input_data.shape[1],self.input_data.shape[2]))
#             self.target_data = np.reshape(self.target_data,(int(self.target_data.shape[0]/10),10,self.target_data.shape[1],self.target_data.shape[2]))
        else:
            self.target_data = target_data_meta[int(0.8 * self.length):]
            self.input_data = input_data_meta[int(0.8 * self.length):]
#             self.input_data = np.reshape(self.input_data,(int(self.input_data.shape[0]/10),10,self.input_data.shape[1],self.input_data.shape[2]))
#             self.target_data = np.reshape(self.target_data,(int(self.target_data.shape[0]/10),10,self.target_data.shape[1],self.target_data.shape[2]))
        self.length = self.input_data.shape[0]
    def __len__(self):
        return 10
#             return self.length

    def __getitem__(self, index):
#         print(self.input_data[index].shape)
        input_data = torch.from_numpy(self.input_data[0])
        target_data = torch.from_numpy(self.target_data[0])
        return input_data, target_data

In [10]:
trainfolder = ['/datadisk/sdc/wangruibo/maml_e2e/140-200-low/dataset_UMa_D=140.0-200.0_&_A=0.0-0.66_s=10000.npy',
            '/datadisk/sdc/wangruibo/maml_e2e/140-200-medium/dataset_UMa_D=140.0-200.0_&_A=0.66-1.33_s=10000.npy',
            '/datadisk/sdc/wangruibo/maml_e2e/140-200-high/dataset_UMa_D=140.0-200.0_&_A=1.33-2.0_s=10000.npy']
testfolder=['/datadisk/sdc/wangruibo/maml_e2e/bit_labels_s_size=10000.npy',
           '/datadisk/sdc/wangruibo/maml_e2e/bit_labels_s_size=10000.npy',
           '/datadisk/sdc/wangruibo/maml_e2e/bit_labels_s_size=10000.npy']

In [11]:
Umadata_train = scratch_dataset(trainfolder,testfolder,mode = 'train')
Umadata_test = scratch_dataset(trainfolder,testfolder,mode = 'val')

In [12]:
trainloader = DataLoader(Umadata_train, batch_size=10, shuffle=False, num_workers=8)
testloader = DataLoader(Umadata_test, batch_size=10, shuffle=False, num_workers=8)

/datadisk/sdc/wangruibo/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
import torch.optim as optim
import time
def e2e_traninig_scratch(net,Epoch,trainloader, testloader,SaveFolder,savename,maml = True,evaluation=False,save_num= 500):
    
    # Construct NN and parameters
#     net = SubCAN(32, 3, 6, 8).cuda()
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in net.parameters()))
    print(NetworkParamNum)
    
    # Construct scheduler
    lr = 0.00001
    # criterion = nn.L1Loss()
    # optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=10**(-5))
    optimizer = optim.Adam(net.parameters(), lr=lr)
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    
    # Initialize some parameters
    min_testloss = 100
    testError  = []
    trainError = []
    costTime   = []
    BER_train = []
    BER_test = []
    KL_train = []
    KL_test = []
    storeStepResult = True
    minloss = 1000000000
    snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 15.0).squeeze()
    zero_loss_train = 0
    zero_loss_eval = 0
    zero_ber_trian = 0
    zero_ber_test = 0
    for dmrs, target in trainloader:
        net.eval()
        target = target.cuda()
        dmrs = dmrs.cuda()
        A = np.flip([[[2**i for i in range(4)]]])
        s = torch.sum(target.cpu()*A, axis=2)
        snr_db = snr_db.cuda()
        s = s.cuda()
        t_pred = net(snr_db,dmrs,s)
        LLR = t_pred
        LLR = torch.where(LLR >= 0, 1,-1)
        LLR = (LLR+1)/2
        LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
        zero_ber_trian += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
        t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
        loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
        
        zero_loss_train += loss.item()
    BER_train.append(zero_ber_trian/len(trainloader))
    trainError.append(zero_loss_train/len(trainloader))
    if evaluation:
        for dmrs, target in testloader:
            net.eval()
            target = target.cuda()
            dmrs = dmrs.cuda()
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(target.cpu()*A, axis=2)
            snr_db = snr_db.cuda()
            s = s.cuda()
            t_pred = net(snr_db,dmrs,s)
            LLR = t_pred
            LLR = torch.where(LLR >= 0, 1,-1)
            LLR = (LLR+1)/2
            LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            zero_ber_test += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
            t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
            zero_loss_eval += loss.item()
        BER_test.append(zero_ber_test/len(testloader))
        testError.append(zero_loss_eval/len(testloader))
    for epoch in range(Epoch):
        
#         randSeed = random.randint(0,10000)
#         if len(ChannelType) == 1:
#             trainloader, testloader = DataSplitter(ChannelType, MaxDopplerList, SNRList, seedNN = randSeed)
#         elif len(ChannelType) > 1:
#             trainloader, testloader = DataSplitterMix(ChannelType, MaxDopplerList, SNRList, seedNN = randSeed) 
        net.train()
        train_loss = 0.0
        test_loss  = 0.0      
        numTrain = 0
        numTest  = 0 
        bertrain = 0
        bertest = 0
        time_start = time.time()
        for dmrs, target in trainloader:
            optimizer.zero_grad()
            target = target.cuda()
            dmrs = dmrs.cuda()
#             logit = dmrs.view(dmrs.shape[0]*dmrs.shape[1],dmrs.shape[2],dmrs.shape[3],dmrs.shape[4])
#             label = target.view(target.shape[0]*target.shape[1],target.shape[2],target.shape[3],target.shape[4])
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(target.cpu()*A, axis=2)
            snr_db = snr_db.cuda()
            s = s.cuda()

            t_pred = net(snr_db,dmrs,s)
            LLR = t_pred
            LLR = torch.where(LLR >= 0, 1,-1)
            LLR = (LLR+1)/2
            LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            bertrain += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
    #             print(t_pred.shape)
            t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
                    # loss = criterion(t_pred, target)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
#         scheduler.step()
            
        net.eval()
        numTrain += len(trainloader)
            
        del t_pred, target, dmrs        
            # 测试
        if evaluation:
            for dmrs, target in testloader:
                target = target.cuda().float()
                dmrs = dmrs.cuda().float()
    #             logit = dmrs.view(dmrs.shape[0]*dmrs.shape[1],dmrs.shape[2],dmrs.shape[3],dmrs.shape[4])
    #             label = target.view(target.shape[0]*target.shape[1],target.shape[2],target.shape[3],target.shape[4])
                A = np.flip([[[2**i for i in range(4)]]])
                s = torch.sum(target.cpu()*A, axis=2)
                snr_db = snr_db.cuda()
                s = s.cuda()

                t_pred = net(snr_db,dmrs,s)
                LLR = t_pred
                LLR = torch.where(LLR >= 0, 1,-1)
                LLR = (LLR+1)/2
                LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
                bertest += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
                t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
        #             print(t_pred.shape)
                loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
#                 KL = KL(t_pred,target)
                        # loss = criterion(t_pred, target)
                test_loss += float(loss.item())
            numTest += len(testloader)
            del t_pred, target, dmrs
        
        time_end = time.time()
        
        train_loss /= numTrain
        bertrain /= numTrain
#         train_loss /= torch.log(torch.tensor([2.]))
#         train_loss = torch.mean(torch.tensor(4.)-torch.sum(train_loss,axis = 3))
        if evaluation:
            test_loss  /= numTest
            bertest /= numTest
            
#         test_loss /= torch.log(torch.tensor([2.]))
        # For print and record Loss
        cost = time_end - time_start
        if evaluation:
            showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][Ber train:%3.5f][Ber test:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss,bertrain,bertest, cost)
        else:
            showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][Ber:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, bertrain, cost)
        print(showStrings, end='\n')
        if maml:
            name = 'MAML' + savename
        else:
            name = 'Joint' + savename
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        BER_train.append(bertrain)
        BER_test.append(bertest)
        file = open(SaveFolder + name + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.write('BER train: ' + str(BER_train) + "\n")
        file.write('BER test: ' + str(BER_test) + "\n")
        file.close()
#         if epoch % save_num == 0:
#             torch.save(net.state_dict(),  SaveFolder+ 'Checkpoints-Epk{}.pth'.format(epoch))

In [ ]:
gs_net = SysGS_torch_gs().cuda()
txt_name = 'CheckPoint'
e2e_traninig_scratch(gs_net,100,trainloader,testloader,'/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/Scratch_10_point/',txt_name,maml = False)

In [ ]:
gs_net = SysGS_torch_gs().cuda()
gs_net.load_state_dict(torch.load('/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/JOINT_ORIGINIAL_5000/Checkpoints-Epk9500.pth'))
txt_name = '_CheckPoint_2000_epoch'
e2e_traninig_scratch(gs_net,100,trainloader,testloader,'/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/Scratch_10_point/',txt_name,maml = False)

In [23]:
gs_net = SysGS_torch_gs().cuda()
gs_net.load_state_dict(torch.load('/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/MAML_ORIGINIAL_5000_no_grad/Checkpoints-Epk2000.pth'))
txt_name = '_CheckPoint_2000_epoch'
e2e_traninig_scratch(gs_net,20,trainloader,testloader,'/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/Scratch_10_point/',txt_name,maml = True)

Total number of paramerters in networks is 4737572  
[Epoch:   1/ 20][TraingLoss: 4.27097][Ber:0.50910][CostTime: 5.08396 s]
[Epoch:   2/ 20][TraingLoss: 4.14147][Ber:0.50908][CostTime: 5.08613 s]
[Epoch:   3/ 20][TraingLoss: 4.00427][Ber:0.50766][CostTime: 4.27403 s]


KeyboardInterrupt: 